In [1]:
import finndex
import datetime

In [2]:
from finndex.sentiment import fearandgreed, trends
from finndex.fundamental import coinmetrics
from finndex.fundamental.coinmetrics import CoinMetricsData
from finndex.util.cryptocurrencies import Cryptocurrencies

In [7]:
from finndex.aggregate.historical import HistoricalSentimentManager
from finndex.aggregate.historical import HistoricalMetricType
manager = HistoricalSentimentManager([HistoricalMetricType.FEAR_AND_GREED, 
                            HistoricalMetricType.TRENDS,
                                     HistoricalMetricType.DAILY_ADDRESSES,
                                     HistoricalMetricType.TRANSACTION_CNT],
                           [Cryptocurrencies.BITCOIN, Cryptocurrencies.ETHEREUM, Cryptocurrencies.TEZOS, 
                            Cryptocurrencies.KYBER_NETWORK, Cryptocurrencies.CHAINLINK], start_date=datetime.datetime.now() - datetime.timedelta(days=5),
                                     end_date=datetime.datetime.now())

In [4]:
historical = manager.get_historical_sentiment()

The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTP

KeyboardInterrupt: 

In [14]:
historical.plot()

In [6]:
prices = manager.get_prices()

In [7]:
print(manager.get_price_correlation())

{(<Cryptocurrencies.KYBER_NETWORK: <finndex.util.cryptocurrencies.Stock object at 0x107b7e5e0>>, -0.2516287980227792), (<Cryptocurrencies.BITCOIN: <finndex.util.cryptocurrencies.Stock object at 0x107b884f0>>, 0.8845651974932542), (<Cryptocurrencies.ETHEREUM: <finndex.util.cryptocurrencies.Stock object at 0x107b881f0>>, 0.5227830476844082), (<Cryptocurrencies.TEZOS: <finndex.util.cryptocurrencies.Stock object at 0x107b88fa0>>, 0.24330847454324367), (<Cryptocurrencies.CHAINLINK: <finndex.util.cryptocurrencies.Stock object at 0x107b7e9d0>>, 0.7313556068709233)}


In [15]:
manager.get_index_sentiment().plot()

In [17]:
index_sentiment = manager.get_index_sentiment()

In [19]:
sentiment_csv = index_sentiment.to_csv()

In [22]:
f = open("finndex.csv", "w")
f.write(sentiment_csv)
f.close()

In [8]:
self = manager
frames = [metric.value(self.start_date, self.end_date, self.currencies_list) for metric in self.keywords_list]

The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.


In [28]:
import pandas as pd
combined = pd.concat(frames, axis=1)

In [30]:
combined[Cryptocurrencies.BITCOIN]

,FearGreed,Trends,AdrActCnt,TxCnt
date,,,,
2020-11-20,NaN,0.586429,NaN,NaN
2020-11-21,0.91,0.560000,0.780587,0.619964
2020-11-22,0.94,0.502917,0.678386,0.549347
2020-11-23,0.90,0.503333,0.786456,0.605287
2020-11-24,0.88,0.662500,0.815583,0.664305
2020-11-25,0.94,0.659091,NaN,NaN


In [64]:
metrics_list = [CoinMetricsData.BLOCK_COUNT, CoinMetricsData.MARKET_CAP, CoinMetricsData.TRANSACTION_CNT, CoinMetricsData.DAILY_ADDRESSES, CoinMetricsData.PRICE_USD]
currencies_list = [Cryptocurrencies.BITCOIN]
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.datetime.now()

In [65]:
cm_data = coinmetrics.get_coinmetrics_dates(metrics_list, start_date, end_date, currencies_list, False, False)

In [43]:
trends_data = trends.get_trends_dates(start_date, end_date, currencies_list)

The request failed: Google returned a response with code 500.


In [46]:
fg_data = fearandgreed.get_fg_dates(start_date, end_date, currencies_list=[Cryptocurrencies.BITCOIN])

In [66]:
combined=pd.concat([cm_data, trends_data,fg_data],axis=1)

In [67]:
# Remove first and last values
combined=combined.drop(combined.tail(1).index)
combined=combined.drop(combined.head(1).index)

In [68]:
combined_csv = combined[Cryptocurrencies.BITCOIN].to_csv()

In [69]:
f2 = open("aggregate.csv", "w")
f2.write(combined_csv)
f2.close()

In [70]:
f2

<_io.TextIOWrapper name='aggregate.csv' mode='w' encoding='UTF-8'>